In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

%load_ext autoreload
%autoreload 2

In [2]:
from windmark.core.schema import Schema, Hyperparameters

In [3]:
Schema(amount='continuous', event='discrete')

In [8]:
params = Hyperparameters(n_fields=31)

In [12]:
params.n_fields

31

In [27]:
pl.DataFrame({'foo': array}).select(pl.struct('foo'))

foo
struct[1]
"{[3, 3, 0]}"
"{[3, 3, 4]}"
"{[3, 0, 2]}"
"{[4, 2, 2]}"
"{[2, 1, 4]}"
"{[0, 0, 1]}"
"{[3, 4, 0]}"
"{[1, 3, 2]}"
"{[0, 1, 1]}"


In [22]:
from enum import IntEnum

class SpecialTokens(IntEnum):

    VAL = 0
    NAN = 1
    UNK = 2
    PAD = 3
    MASK = 4

In [23]:
int(SpecialTokens.PAD)

3

In [24]:
len(SpecialTokens)

5